# Throughtput Benchmarking  Seldon-Core on GCP Kubernetes

The notebook will provide a benchmarking of seldon-core for maximum throughput test. We will run a stub model and test using REST and gRPC predictions. This will provide a maximum theoretical throughtput for model deployment in the given infrastructure scenario.

## Create Cluster

Create a cluster in GKE. You can use GKE console or `gcloud` command line.

## Setup Seldon Deployment

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

In [88]:
DEPLOYMENT_NAME="seldon-loadtest"
NAMESPACE=""
GCP_PROJECT=""
LOCUST_IMAGE_LOCATION=""  # I believe for the Seldon team it is loadtest-seldon-core-loadtest:5001
MODEL_IMG_TAG=1.0
ROUTER_IMG_TAG=1.1

!gcloud -q container clusters get-credentials $GCP_PROJECT --region us-east1 --project ld-shipyard
!helm template $DEPLOYMENT_NAME model --values model/values.yaml --set img_tag.model=$MODEL_IMG_TAG --set img_tag.router=$ROUTER_IMG_TAG | kubectl apply -n $NAMESPACE -f -

Fetching cluster endpoint and auth data.
kubeconfig entry generated for prod-nezbleu.
seldondeployment.machinelearning.seldon.io/loadtest created


## REST 
Run REST benchmark

In [ ]:
!helm install loadtest locust  \
    --namespace $NAMESPACE \
    --set image.name=$LOCUST_IMAGE_LOCATION \
    --set locust.script=mab_rest_locust.py \
    --set locust.host=http://loadtest-loadtest-dummy-model:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=4 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set locust.payloadPath='/home/seldon/resources/payload-single-model.json' \
    --set replicaCount=4

Get stats from locust running on GKE

In [91]:
#This should pass
!kubectl logs $(kubectl get pod -l name=locust-master-1 -n $NAMESPACE -o NAME) -n $NAMESPACE -c locust-master-1 --tail=12

 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s failures/s
--------------------------------------------------------------------------------------------------------------------------------------------
 POST predictions                                                7924 1926(24.31%)      18       4   10008  |      12  304.20   31.20
--------------------------------------------------------------------------------------------------------------------------------------------
 Aggregated                                                      7924 1926(24.31%)      18       4   10008  |      12  304.20   31.20

 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s failures/s
--------------------------------------------------------------------------------------------------------------------------------------------
 POST predictions                 

In [83]:
!helm uninstall loadtest --namespace $NAMESPACE

release "loadtest" uninstalled


In [ ]:
!helm install loadtest locust  \
    --namespace $NAMESPACE \
    --set image.name=$LOCUST_IMAGE_LOCATION \
    --set locust.script=mab_rest_locust.py \
    --set locust.host=http://loadtest-loadtest-dummy-model:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=16 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set locust.payloadPath='/home/seldon/resources/payload-single-model.json' \
    --set replicaCount=4

In [59]:
# Now it hangs, this is shown by 0 requests being sent per second, 0 errors, you can try calling the API outside of locust to verify
!kubectl logs $(kubectl get pod -l name=locust-master-1 -n $NAMESPACE -o NAME) -n $NAMESPACE -c locust-master-1 --tail=12

 Aggregated                                                         0     0(0.00%)       0       0       0  |       0    0.00    0.00

 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s failures/s
--------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------
 Aggregated                                                         0     0(0.00%)       0       0       0  |       0    0.00    0.00

 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s failures/s
--------------------------------------------------------------------------------------------------------------------------------------------
---------------------------------

In [86]:
!helm uninstall loadtest --namespace $NAMESPACE

release "loadtest" uninstalled


In [87]:
!kubectl delete sdep loadtest -n $NAMESPACE

seldondeployment.machinelearning.seldon.io "loadtest" deleted
